# Estimate Tests

In [1]:
# For config file
import json

import pandas as pd
import sqlalchemy as sql

from pathlib import Path

In [2]:
DDAM = sql.create_engine('mssql+pymssql://DDAMWSQL16')

CONFIG = None
with open("./config.json", "r") as f:
    CONFIG = json.load(f)

## Aggregation by Geography Function

In [3]:
def get_table_by_geography(connection, config, est_table, geo_level, 
    est_name="2020_06",
    pivot=False, 
    debug=False):
    """
    Get the input estimates table grouped by the input geography level

    :param connection:  The connection to the relevant SQL server (AFAIK always DDAMWSWL16)
    :param config:      The config file. See "./config.json" for details
    :param est_table:   The name of the estimates table. This is the part after "dw_"
    :param geo_level:   The geography level to group by. This is a string input corresponding to one
                        of the column names of [demographic_warehouse].[dim].[mgra_denormalize]. For
                        example, this variable could contain "sra", "college", or "jurisdiction"
    :param est_name:    Which estimates table to pull from. See the variable EST_BASE_TABLE for
                        more details on how this variable is used
    :param pivot:       By default, False. If True, change the format of the table from being tall
                        to wide. For more details, see the bottom of the function for exactly what 
                        is going on
    :param debug:       By default, False. If True, then print out diagnostic statements including
                        the complete SQL query used
    :returns:           Dataframe containing the requested table grouped by the geography level
    """
    # This variable is used to deal with the unique behavior of households table. We ignore the 
    # household_size_id column and just group by the geography level
    households = (est_table == "households")

    # The basic format of every table we are looking at. To use, call
    # EST_BASE_TABLE.format(<TABLE NAME>)
    EST_BASE_TABLE = "[estimates].[est_" + est_name + "].[dw_{0}]"

    # The basic format of every dim table we are looking at. To use, call:
    # DIM_BASE_TABLE.format(<TABLE NAME>)
    DIM_BASE_TABLE = "[demographic_warehouse].[dim].[{0}]"

    # Create the basic structure of the SQL query
    # Note, none of the formatted strings should end with a ","
    query = """
SELECT {mgra_denormalize_col}, yr_id, {dim_named_cols}, {agg_col}
FROM {est_base_table} as tbl
{joins}
WHERE {geography_filter}
GROUP BY {mgra_denormalize_col}, yr_id, {join_col}, {dim_named_cols}
ORDER BY {mgra_denormalize_col}, yr_id, {join_col}
"""
    if(households):
        # In the households table, we ignore the hosueholds_size_id column, which means we only have
        # to join with mgra_denormalize
        query = """
SELECT {mgra_denormalize_col}, yr_id, {agg_col}
FROM {est_base_table} as tbl
{joins}
WHERE {geography_filter}
GROUP BY {mgra_denormalize_col}, yr_id
ORDER BY {mgra_denormalize_col}, yr_id
"""

    # We additionally need the columns that exist in the estimates table
    COLUMNS = pd.read_sql_query(f"""
        SELECT TOP(0) *
        FROM {EST_BASE_TABLE.format(est_table)}
    """, con=connection).columns
    if(debug):
        print(f"{'Columns in estimates table:' : <32}", list(COLUMNS))
        print()

    # From the list of columns, we can find exactly which columns we want to be joining on. These
    # are the columns which end with "_id" but are not "mgra_id" nor "yr_id"
    ID_COLUMNS = [col for col in COLUMNS if 
        col.endswith("_id") and 
        col != "mgra_id" and 
        col != "yr_id"
    ]
    if(households):
        ID_COLUMNS = []

    # The field {mgra_denormalize_col} is asking for the column name that contains the geography
    # variable ("sra", "college", "jurisdiction", etc.)
    mgra_denormalize_col = geo_level

    # The field {dim_named_cols} is asking for the (formatted) columns in the dim tables that 
    # contain the long form representations of the ids. For example, in the dim table age_group,
    # age_group_id=1 corresponds to name="Under 5", so we want the "name" column as it is the most
    # descriptive
    dim_named_cols = ""
    if(not households):
        for id_col in ID_COLUMNS:
            dim_named_cols += f"{config['dim'][id_col]['dim_table']}.{config['dim'][id_col]['column(s)'][0]}, "
        dim_named_cols = dim_named_cols[:-2] # Remove the trailing comma
    
    # The field {agg_col} is asking for the column of the estimates table we are aggregating on
    # and the function used to aggregate. This information is contained in config["est"]
    agg_list = config["est"][est_table]
    agg_col = ""
    for aggregation in agg_list:
        agg_col += "{function}({col}) as {col}".format(function=aggregation[1], col=aggregation[0])
        agg_col += ", "
    agg_col = agg_col[:-2] # Remove the trailing comma
    if(debug):
        print(f"{'Aggregation instructions:' : <32}", config["est"][est_table])
        print()
    
    # The field {est_base_table} is asking for
    est_base_table = EST_BASE_TABLE.format(est_table)
    
    # The field {joins} is asking for formatted list of INNER JOINs that add on each dim table to
    # the estimates table. This information is contained in config["dim"]
    # TODO: Are there null mgra_id values? May need to change to LEFT JOIN
    # Note, we always want to join on mgra_id, so add that to the list
    JOIN_COLS = ["mgra_id"] + ID_COLUMNS
    if(households):
        JOIN_COLS = ["mgra_id"]
    joins = ""
    for join_col in JOIN_COLS:
        dim_table = config["dim"][join_col]["dim_table"]
        joins += f"""
INNER JOIN {DIM_BASE_TABLE.format(dim_table)} as {dim_table} ON
    {dim_table}.{join_col} = tbl.{join_col}
"""
    if(debug):
        print(f"{'Columns to join on:' : <32}", list(JOIN_COLS))
        print()
    
    # The field {geography_filter} is asking for the conditional where we only get the rows of the 
    # table where the geography level we are interested in is not NULL
    geography_filter = f"{mgra_denormalize_col} IS NOT NULL"

    # The field {join_col} is asking for the column of the estimates table we are joining on in 
    # order to keep categorical variables in the same order
    # TODO: This assumes there is only one join to be made
    join_col = ""
    if(not households):
        join_col = f"tbl.{ID_COLUMNS[0]}"

    # Fill in the blanks of the query
    if(not households):
        query = query.format(
            mgra_denormalize_col=mgra_denormalize_col,
            dim_named_cols=dim_named_cols,
            agg_col=agg_col,
            est_base_table=est_base_table,
            joins=joins,
            geography_filter=geography_filter,
            join_col = join_col
        )
    else:
        query = query.format(
            mgra_denormalize_col=mgra_denormalize_col,
            agg_col=agg_col,
            est_base_table=est_base_table,
            joins=joins,
            geography_filter=geography_filter,
            join_col = join_col
        )
    if(debug):
        print("*** FULL QUERY BELOW ***")
        print(query)
        print("*** END FULL QUERY ***")

    # Get the table into pandas
    table = pd.read_sql_query(query, con=connection)

    # Pivot the table if requested
    if(pivot):
        # For every table, there are 1-3 categorical columns, and 1-4 value columns. Each unique
        # combination of all categorical columns and one value column will form a new column

        # First, create the list of index columns, categorical column(s), and value column(s)
        IND_COLS = [geo_level, "yr_id"]
        CAT_COLS = [config["dim"][col]["column(s)"][0] for col in ID_COLUMNS]
        VAL_COLS = [col[0] for col in config["est"][est_table]]

        # Custom behavior for the age_sex_ethnicity table
        if(est_table == "age_sex_ethnicity"):
            IND_COLS += ["name", "sex"]
            CAT_COLS = ["long_name"]

        # Pivot the table
        table = table.pivot_table(
            index=IND_COLS, 
            columns=CAT_COLS,
            values=VAL_COLS,
            aggfunc=sum) # Not used except for age_sex_ethnicity table

    # Return the table
    return table

In [4]:
# Example usage
get_table_by_geography(DDAM, CONFIG, "age", "jurisdiction")

,jurisdiction,yr_id,name,population
0,Carlsbad,2010,Under 5,6236
1,Carlsbad,2010,5 to 9,7225
2,Carlsbad,2010,10 to 14,7351
3,Carlsbad,2010,15 to 17,4374
4,Carlsbad,2010,18 and 19,2047
...,...,...,...,...
4175,Vista,2020,65 to 69,4508
4176,Vista,2020,70 to 74,3478
4177,Vista,2020,75 to 79,2366
4178,Vista,2020,80 to 84,1706


## Consolidated Files

In [5]:
def consolidate(connection, config,
    est_name="2020_06",
    geo_list=["region", "jurisdiction", "cpa"], 
    est_table_list=["age", "ethnicity", "household_income", "households", "housing", "population", "sex"],
    save=False,
    save_folder=None):
    """
    Conoslidate the input estimates tables column wise for each input geography. This function 
    returns one dataframe per geography level, as opposed to combining everything together

    :param connection:      The connection to the relevant SQL server (AFAIK always DDAMWSWL16)
    :param config:          The config file. See "./config.json" for details
    :param est_name:        Which estimates table to pull from. See the variable EST_BASE_TABLE in 
                            the function get_table_by_geography for more details
    :param geo_list:        The geographies to cosolidate along.
    :param est_table_list:  Whcih estimates tables we want to consolidate
    :param save:            False by default. If False, then only return the consolidated tables. If 
                            True, then use save_folder to save the consolidated tables and return
                            the tables
    :param save_folder:     The folder in which to save consolidated files. Only used if save=True
    :returns:               A list containing the consolidated tables (dataframes) in the order of 
                            geo_list
    """
    # Store each cosolidated table by geography level here
    combined_tables = []

    # Loop over the geography levels we want to consolidate on
    for geo in geo_list:

        # Each estimate table will create one df each of which has the same number of rows (one row
        # per unique geography region and year). Store them here to merge after
        est_tables = []

        # Loop over every estimate table we want to consolidate
        for est_table_name in est_table_list:

            # Get the estimate table
            est_table = get_table_by_geography(connection, config, est_table_name, geo, 
                est_name=est_name, pivot=True)

            # Do some transformations to align the format with what we want in the csv
            # Similar to in get_table_by_geography, we have different behavior for the households
            # table as we ignore the column household_size_id. As a result, the table returned by
            # get_table_by_geography is already in the correct format
            est_table = est_table.reset_index()
            if(est_table_name != "households"):
                # TODO: Possible bug when consolidating age_sex_ethncity table relating to usage
                # of hardcoded number 2 below
                est_table.columns = est_table.columns.get_level_values(0)[:2].append(
                    est_table.columns.get_level_values(1)[2:])

            # Add the transformed estimate table to our list of tables
            est_tables.append(est_table)

        # Combine all the transformed estimate tables into one large table
        combined_table = pd.concat(est_tables, axis=1)

        # Since each of the estimates table has its own version of geo, "yr_id", remove those
        # duplicate columns
        combined_table = combined_table.loc[:, ~combined_table.columns.duplicated()]

        # Store the combined table
        combined_tables.append(combined_table)

        # Save the table if requested
        if(save):
            # Save each table using the geography level to distinguish
            file_name = f"consolidated_{est_name}_{geo}_QA.csv"
            combined_table.to_csv(save_folder / file_name, index=False)
            
    # Return all the combined tables
    return combined_tables

In [6]:
# Example usage
# Note, geo_list and est_table_list are optional parameters as they both have default values
# Also, this is only showing the jurisdiction table (index [1])
consolidate(DDAM, CONFIG)[1]

,jurisdiction,yr_id,10 to 14,15 to 17,18 and 19,20 to 24,25 to 29,30 to 34,35 to 39,40 to 44,...,Single Family - Detached,Single Family - Multiple Unit,Single-family Attached,Single-family Detached,Group Quarters - College,Group Quarters - Military,Group Quarters - Other,Household Population,Female,Male
0,Carlsbad,2010,7351,4374,2047,4806,5986,6348,7595,8248,...,22004,5934,0,0,0,0,915,104413,53736,51592
1,Carlsbad,2011,7372,4510,2464,5387,6025,6464,7269,8321,...,22096,5995,0,0,0,0,915,105853,54437,52331
2,Carlsbad,2012,7276,4650,2675,6018,6009,6626,6837,8451,...,22263,6044,0,0,0,0,915,107357,55193,53079
3,Carlsbad,2013,7229,4684,2721,6593,6102,6779,6731,8377,...,22517,6088,0,0,0,0,915,108740,55752,53903
4,Carlsbad,2014,7049,4724,2718,7068,6121,6876,6747,8080,...,22823,6151,0,0,0,0,915,110202,56456,54661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,Vista,2016,7259,4671,2952,7488,6683,7125,6081,5853,...,14712,2251,0,0,0,0,2051,97084,49492,49643
205,Vista,2017,7664,4782,2903,8165,6819,7068,6339,5810,...,14834,2259,0,0,0,0,2059,99820,50811,51068
206,Vista,2018,7860,4851,2893,7867,6819,6779,6570,5745,...,14917,2271,0,0,0,0,2123,99861,50789,51195
207,Vista,2019,8038,4810,2939,7933,6699,6513,6719,5756,...,14941,2274,0,0,0,0,2142,99956,50918,51180


In [7]:
# # Example usage with saving
# # Note: Commented out to avoid accidentally running
# user = "eli"
# save_location = Path(f"C:/Users/{user}/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/")
# _ = consolidate(DDAM, CONFIG, save=True, save_folder=save_location) # "_ = " to ignore return values

## Individual Files

In [8]:
def individual(connection, config,
    est_name="2020_06",
    geo_list=["region", "jurisdiction", "cpa"], 
    est_table_list=["age", "ethnicity", "household_income", "age_sex_ethnicity"],
    eth_by_age=True,
    save=False,
    save_folder=None):
    """
    Generate individual estimates tables for each input geography. This function returns one
    dataframe for each geography level / estimate table. Because of the way looping is done, the 
    order of dfs is first geo_level each estimate table, second geo_level each estimate table, etc.

    :param connection:      The connection to the relevant SQL server (AFAIK always DDAMWSWL16)
    :param config:          The config file. See "./config.json" for details
    :param est_name:        Which estimates table to pull from. See the variable EST_BASE_TABLE in 
                            the function get_table_by_geography for more details
    :param geo_list:        The desired geographies
    :param est_table_list:  The desired estimates tables
    :param eth_by_age:      True by default. If True, then additionally get the estimates table 
                            age_sex_ethnicity, but grouped by only age. If False, then don't. This
                            is not very extensible, but oh well :(
    :param save:            False by default. If False, then only return the individual tables. If 
                            True, then use save_folder to save the individual tables and return
                            the tables
    :param save_folder:     The folder in which to save individual files. Only used if save=True
    :returns:               A list containing the individual tables (dataframes) in the order of 
                            geo_list x (est_table_list + eth_by_age)
    """
    # Store each individual table by geography level x est_table_list here
    individual_tables = []

    # Loop over the geography levels we want to get individual files on
    for geo in geo_list:

        # Loop over every estimate table we want to get
        for est_table_name in est_table_list:

            # Get the estimate table
            est_table = get_table_by_geography(connection, config, est_table_name, geo, 
                est_name=est_name, pivot=True)

            # Do some transformations to align the format with what we want in the csv
            # Similar to in get_table_by_geography, we have different behavior for the households
            # table as we ignore the column household_size_id. As a result, the table returned by
            # get_table_by_geography is already in the correct format
            est_table = est_table.reset_index()
            if(est_table_name != "households"):
                column_name_pivot_point = list(est_table.columns.get_level_values(0)).index(
                    config["est"][est_table_name][0][0])
                est_table.columns = est_table.columns.get_level_values(0)[:column_name_pivot_point].append(
                    est_table.columns.get_level_values(1)[column_name_pivot_point:])

            # Store the individual table
            individual_tables.append(est_table)

            # Save the table if requested
            if(save):
                # Save each table using the geography level to distinguish
                file_name = f"{est_name}_{est_table_name}_{geo}_QA.csv"
                est_table.to_csv(save_folder / file_name, index=False)

        # Check if we additionally want to get ethnicity broken down by only age (not gender)
        if(eth_by_age):
            # Get the age_sex_ethnicity table
            est_table = get_table_by_geography(connection, config, "age_sex_ethnicity", geo, 
                est_name=est_name, pivot=True).reset_index()

            # Do the same transforms
            column_name_pivot_point = list(est_table.columns.get_level_values(0)).index("population")
            est_table.columns = est_table.columns.get_level_values(0)[:column_name_pivot_point].append(
                est_table.columns.get_level_values(1)[column_name_pivot_point:])

            # Group by every categorical variable except sex
            est_table = est_table.groupby([geo, "yr_id", "name"]).sum().reset_index()

            # Store the individual table
            individual_tables.append(est_table)

            # Save the table if requested
            if(save):
                # Save each table using the geography level to distinguish
                file_name = f"{est_name}_age_ethnicity_{geo}_QA.csv"
                est_table.to_csv(save_folder / file_name, index=False)
            
    # Return all the combined tables
    return individual_tables

In [9]:
# Example usage
x = individual(DDAM, CONFIG, geo_list=["region", "jurisdiction"], 
    est_table_list=["age", "ethnicity"], eth_by_age=True)
x[0]

,region,yr_id,10 to 14,15 to 17,18 and 19,20 to 24,25 to 29,30 to 34,35 to 39,40 to 44,...,50 to 54,55 to 59,60 and 61,62 to 64,65 to 69,70 to 74,75 to 79,80 to 84,85 and Older,Under 5
0,San Diego,2010,198716,128000,97095,270750,250737,220185,211012,209551,...,210980,180305,64088,85223,103241,77313,64347,52564,53960,203423
1,San Diego,2011,198866,128833,95984,269851,246589,224827,209095,211848,...,213219,185244,65395,90092,108198,80080,64763,53044,55169,209818
2,San Diego,2012,198403,131596,97211,267545,240615,230265,207600,213448,...,215234,191186,66248,94222,116570,83765,65707,53304,56304,215677
3,San Diego,2013,199295,133282,98736,265080,234627,235654,208516,213747,...,216585,196575,68049,96238,125993,88669,67357,53490,57184,219753
4,San Diego,2014,200875,132669,100101,263770,228231,240016,210707,212562,...,217419,200838,71049,97916,134674,93526,69690,53723,58123,222064
5,San Diego,2015,202795,131361,101520,263884,222382,240792,214787,210570,...,216930,204873,74316,99943,143500,98097,72596,54019,59542,223824
6,San Diego,2016,205244,131064,100003,263975,218370,236207,219858,207477,...,213627,207778,76690,102855,150812,103135,75248,54291,60744,223355
7,San Diego,2017,210062,131314,98316,264924,215777,230157,224897,205662,...,209507,209560,78084,107170,155161,110605,78080,55166,61590,221684
8,San Diego,2018,215718,131683,98570,264797,212733,223717,229591,205794,...,205816,210362,79111,111109,157987,119302,81773,56630,62017,219100
9,San Diego,2019,220580,132502,99494,263897,210274,216586,233121,207126,...,202841,210308,79972,113816,161640,127146,85638,58576,63080,213921


In [10]:
# # Example usage with saving
# # Note: Commented out to avoid accidentally running
# user = "eli"
# save_location = Path(f"C:/Users/{user}/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-56 Estimates 2021 QC/Data/")
# _ = individual(DDAM, CONFIG, save=True, save_folder=save_location) # "_ = " to ignore return values